In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
#carga del archivos con pandas
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BD/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BD/test.csv')
df_test

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Si,Si,N,Si,Si,Si,Primaria incompleta
296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa
296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Educación profesional completa
296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa


In [4]:
df_test.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE'],
      dtype='object')

In [5]:
k = df_test.isna().sum()
k[k!=0]

ESTU_VALORMATRICULAUNIVERSIDAD     2723
ESTU_HORASSEMANATRABAJA           13379
FAMI_ESTRATOVIVIENDA              13795
FAMI_TIENEINTERNET                11539
FAMI_EDUCACIONPADRE                9993
FAMI_TIENELAVADORA                17259
FAMI_TIENEAUTOMOVIL               18918
ESTU_PAGOMATRICULAPROPIO           2807
FAMI_TIENECOMPUTADOR              16439
FAMI_TIENEINTERNET.1              11539
FAMI_EDUCACIONMADRE               10223
dtype: int64

In [6]:
# Columnas con datos faltantes
columns_with_missing_values = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
    'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA',
    'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1',
    'FAMI_EDUCACIONMADRE'
]

In [7]:
# Rellenar valores faltantes con la moda de cada columna
for column in columns_with_missing_values:
    mode_value = df_test[column].mode()[0]
    df_test[column].fillna(mode_value, inplace=True)

In [8]:
k = df_test.isna().sum()
k[k!=0]

Series([], dtype: int64)

In [9]:
columns_to_exclude = ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO']
# Obtener las columnas que se van a codificar (todas menos las excluidas)
columns_to_encode = [col for col in df_test.columns if col not in columns_to_exclude]
columns_to_encode

['ESTU_PRGM_DEPARTAMENTO',
 'ESTU_VALORMATRICULAUNIVERSIDAD',
 'ESTU_HORASSEMANATRABAJA',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_TIENEINTERNET',
 'FAMI_EDUCACIONPADRE',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEAUTOMOVIL',
 'ESTU_PRIVADO_LIBERTAD',
 'ESTU_PAGOMATRICULAPROPIO',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENEINTERNET.1',
 'FAMI_EDUCACIONMADRE']

In [10]:
# Aplicar la codificación one-hot a las columnas seleccionadas
df_test_pro = pd.get_dummies(df_test[columns_to_encode]).astype(int)
df_test_pro

,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
296782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296783,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296784,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Mapear la columna target a valores numéricos en el conjunto de entrenamiento
df_train['RENDIMIENTO_GLOBAL'] = df_train['RENDIMIENTO_GLOBAL'].map({
    "bajo": 1, "medio-bajo": 2, "medio-alto": 3, "alto": 4
})

In [12]:
# Crear el TargetEncoder
encoder = TargetEncoder(cols=['ESTU_PRGM_ACADEMICO'])

# Ajustar el encoder en el conjunto de entrenamiento
encoder.fit(df_train.drop('RENDIMIENTO_GLOBAL', axis=1), df_train['RENDIMIENTO_GLOBAL'])

# Transformar el conjunto de datos de prueba
df_test_encoded = encoder.transform(df_test)
df_test_encoded

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,2.010169,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,2.167702,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,3.093806,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,2.193853,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,2.298813,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,2.298813,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Si,Si,N,Si,Si,Si,Primaria incompleta
296782,209415,20183,2.629461,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa
296783,239074,20212,2.629461,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Educación profesional completa
296784,963852,20195,3.298479,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa


In [13]:
# Concatenar con las columnas no codificadas
df_test_pro = pd.concat([df_test_encoded[['ID','PERIODO','ESTU_PRGM_ACADEMICO']], df_test_pro], axis=1)
df_test_pro

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,550236,20183,2.010169,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,98545,20203,2.167702,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,499179,20212,3.093806,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,782980,20195,2.193853,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,785185,20212,2.298813,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,2.298813,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
296782,209415,20183,2.629461,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296783,239074,20212,2.629461,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
296784,963852,20195,3.298479,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Crear el escalador Min-Max
scaler = MinMaxScaler()

# Normalizar las columnas
df_test_pro['PERIODO'] = scaler.fit_transform(df_test_pro[['PERIODO']])
df_test_pro['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(df_test_pro[['ESTU_PRGM_ACADEMICO']])
df_test_pro

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,550236,0.000000,0.286217,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,98545,0.666667,0.344421,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,499179,0.966667,0.686593,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,782980,0.400000,0.354084,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,785185,0.966667,0.392864,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,0.400000,0.392864,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
296782,209415,0.000000,0.515029,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296783,239074,0.966667,0.515029,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
296784,963852,0.400000,0.762215,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_test_pro.to_csv('/content/drive/MyDrive/Colab Notebooks/BD/test_procesado.csv', index=False)